In [25]:
%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np

# Define parameters
INF = 100 # out-of-bounds value

# Define functions and derivatives
def d_sin(x, n=0):
    """n-th derivative of sin(x)."""
    
    r = n % 4
    if r == 0:
        return np.sin(x)
    elif r == 1:
        return np.cos(x)
    elif r == 2:
        return -np.sin(x)
    elif r == 3:
        return -np.cos(x)

def d_cos(x, n=0):
    """n-th derivative of cos(x)."""
    
    return d_sin(x, n=n+1)

def d_exp(x, n=0):
    """n-th derivative of exp(x)."""
    
    return np.exp(x)

def d_log(x, n=0):
    """n-th derivative of log(x+1)."""
    
    if n == 0:
        return np.log(x+1)
    else:
        return (np.math.factorial(n-1) * (-1)**(n+1)) / (x+1)**n

# Define a function name dictionary
func = {}
func["sin(x)"] = d_sin
func["cos(x)"] = d_cos
func["exp(x)"] = d_exp
func["log(x+1)"] = d_log

# Define Taylor polynomial
def taylor(x, fname, a, n):
    """Taylor polynomial for a given function.
    
    Positional arguments:
    x - input value
    fname - key from 'func' dictionary
    a - center
    n - polynomial degree
    """
    
    out = 0.0 # output value
    
    # Add terms of Taylor polynomial
    for i in range(n+1):
        out += (func[fname](a, n=i) / np.math.factorial(i)) * (x-a)**i
    
    return out

# Set up plot
fig, ax = plt.subplots()
xbase = np.linspace(-6.2, 6.2, 101) # base x-values

# Draw plot lines
@widgets.interact(fname=func.keys(), a=(-6.2, 6.2, 0.05), n=(0, 10, 1))
def update1(fname="sin(x)", a=0.0, n=1):
    
    global ax
    a0 = a
    
    # Generate function values
    if fname == "log(x+1)":
        x = np.linspace(-0.99, 6.2, 71)
        a0 = max(a0, -0.9)
    else:
        x = np.linspace(-6.2, 6.2, 101)
    y = np.zeros_like(x)
    
    # Redraw plot
    ax.clear()
    ax.set_xlim([-6.2, 6.2])
    ax.set_ylim([-4, 4])
    plt.title(fname)
    ax.grid(False)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.plot(x, func[fname](x), color="C0")
    y = taylor(xbase, fname, a0, n)
    ax.plot(xbase, y, color="C1")
    ax.plot(a0, func[fname](a0), color="C1", marker=".", markersize=10)
    if fname == "log(x+1)":
        ax.plot([-1, -1], [-INF, INF], color="black", linestyle="dashed")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(Dropdown(description='fname', options=('sin(x)', 'cos(x)', 'exp(x)', 'log(x+1)'), value=…